# Parse the excel spreadsheet into a tidy format

In [1]:
import os
import itertools

import pandas
import numpy

In [2]:
# publishers in order of how they appear in the spreadsheet
publishers = [
    'Elsevier',
    'Wiley',
    'Springer',
    'Taylor & Francis',
    'Sage',
    'Oxford University Press',
    'Cambridge University Press',
    'Nature Publishing Group',
    'Royal Society of Chemistry',
    'Institute of Physics Publishing',
]

# years in order of how they appear in the spreadsheet
years = list(range(2010, 2015))

In [3]:
url = 'http://files.figshare.com/2096947/Journal_publishing_cost_FOIs_UK_universities.xlsx'
! wget --directory-prefix=download/ --timestamping $url

--2015-06-05 13:59:17--  http://files.figshare.com/2096947/Journal_publishing_cost_FOIs_UK_universities.xlsx
Resolving files.figshare.com (files.figshare.com)... 54.231.129.52
Connecting to files.figshare.com (files.figshare.com)|54.231.129.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121697 (119K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Server file no newer than local file ‘download/Journal_publishing_cost_FOIs_UK_universities.xlsx’ -- not retrieving.



In [4]:
# read excel spreadsheet from figshare
path = os.path.join('download', 'Journal_publishing_cost_FOIs_UK_universities.xlsx')
cost_df = pandas.read_excel(path, sheet='Responses', skiprows=1)[:-2]

# add column names
id_vars = ['university', 'total']
value_vars = ['{} | {}'.format(p, y) for p, y in itertools.product(publishers, years)]
cost_df.columns = id_vars + value_vars

# Round total costs
cost_df.total = numpy.around(cost_df.total, decimals=2)

# gather data into a tidy format
cost_df = pandas.melt(cost_df, id_vars=id_vars, var_name='publisher_year', value_name='cost')

# drop observations with missing values
cost_df = cost_df.dropna()

# separate the combined publisher and year column 
py_df = cost_df.publisher_year.str.split(r' \| ', expand=True)
py_df.columns = ['publisher', 'year']
cost_df = py_df.join(cost_df)
cost_df = cost_df.drop('publisher_year', axis=1)

# convert cost to date
cost_df.year = cost_df.year.convert_objects(convert_dates=True)

In [5]:
# show rows with invalid costs
converted_cost = cost_df.cost.convert_objects(convert_numeric=True)
cost_df[converted_cost.isnull()]

,publisher,year,university,total,cost
4327,Oxford University Press,2013,University of Essex,1796512.34,"£30,089,45"
4742,Cambridge University Press,2010,York St John University,769711.00,£3 631


In [6]:
# remove rows with invalid costs
cost_df.cost = converted_cost
cost_df.dropna()
cost_df = cost_df.reset_index(drop=True)
cost_df.tail()

,publisher,year,university,total,cost
5887,Institute of Physics Publishing,2014,University of West London,61817.41,0
5888,Institute of Physics Publishing,2014,University of Winchester,953812.42,0
5889,Institute of Physics Publishing,2014,Writtle College,0.00,0
5890,Institute of Physics Publishing,2014,University of York,6142532.23,46345
5891,Institute of Physics Publishing,2014,York St John University,769711.00,0


In [7]:
# save tidy dataframe as a tsv
cost_df.to_csv(os.path.join('data', 'subscriptions.tsv'), index=False, sep='\t')